# MATH2319 Machine Learning
Semester 1, 2020

Assignment 2


`I hereby agree to follow any and all assignment rules and procedures as stated in Canvas for this course, MATH2319. In particular, I solemnly swear that I will not discuss/ have not discussed my assignment solutions with anyone in any way and the solutions I am submitting are my own personal work.`


*Arnav Nambiar* - 3729631

# Question1


We will be building a simple decision tree with depth 1 to predict the 'annual income' target feature. We will use the Gini Index split criterion to this end.


**Importing Required Libraries:**

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) 

**Reading the data;**

In [3]:
q1 = pd.read_csv("A2_Q1.csv")

In [4]:
print('The data set has '+ str(q1.shape[1])+' columns and '+ str(q1.shape[0])+ ' rows.')

The data set has 6 columns and 20 rows.


To check the first 5 rows of the dataset;

In [5]:
q1.head()

,ID,Age,Education,Marital_Status,Occupation,Annual_Income
0,1,39,bachelors,never married,professional,high
1,2,50,doctorate,married,professional,mid
2,3,18,high school,never married,agriculture,low
3,4,30,bachelors,married,professional,mid
4,5,37,high school,married,agriculture,mid


## Part A

Here, the impurity of the target feature will be calculated using Gini Index.

We first compute the probability of each categories in the target.

In [6]:
q1.iloc[:,5].value_counts()

mid     12
high     5
low      3
Name: Annual_Income, dtype: int64

The probabilities for target feature;

In [7]:
mid = 12 / 20
high = 5 / 20
low = 3 / 20

print('The probability of annual income being low is ' + str(low)+ ', mid is '+str(mid)+', and high is '+str(high))

The probability of annual income being low is 0.15, mid is 0.6, and high is 0.25


The Gini index is how often the target levels of instances in
a dataset would be misclassified if predictions were made based only on the distribution of
the target levels in the dataset.

$$ Gini Index =1 - \sum_l^{} P(t=1)^2$$

In [8]:
GiniIndex = 1 - ((mid)**2 +  (high)**2  + (low)**2)

In [9]:
GiniIndex

0.5549999999999999

The Gini impurity index rounded to 3 decimals it is 0.555.

## Part B

Here, the root node will be determined for the decision tree algorithms. WE begin by computing the Partition Gini Indexes, Information Gains, and the remainders.

### Impurity and Information Gains for Categorical Variables

We drop the Age and ID column as the Age column is the only continuous descriptive feature and the ID column is just a unique identifier.

In [10]:
catdata = q1.drop(columns=['Age','ID'])
catdata.head()

,Education,Marital_Status,Occupation,Annual_Income
0,bachelors,never married,professional,high
1,doctorate,married,professional,mid
2,high school,never married,agriculture,low
3,bachelors,married,professional,mid
4,high school,married,agriculture,mid




To define the Computation part of Gini Impurity function;

In [11]:
def compute_impurity(feature, impurity_criterion):
   
    probs = feature.value_counts(normalize=True)
    
    if impurity_criterion == 'entropy':
        impurity = -1 * np.sum(np.log2(probs) * probs)
    elif impurity_criterion == 'gini':
        impurity = 1 - np.sum(np.square(probs))
    else:
        raise ValueError('Unknown impurity criterion')
        
    return(round(impurity, 3))

print('impurity using gini index:', compute_impurity(catdata.iloc[:,3], 'gini'))


impurity using gini index: 0.555


The Gini Index is 0.555, which verifies the Gini Index found in the previous section.

The Information Gain function;

In [12]:
def comp_feature_information_gain(df, target, descriptive_feature, split_criterion):  
    print('target feature:', target)
    print('descriptive_feature:', descriptive_feature)
    print('split criterion:', split_criterion)
            
    target_entropy = compute_impurity(df[target], split_criterion)
    entropy_list = list()
    weight_list = list()
    
    for level in df[descriptive_feature].unique():
        df_feature_level = df[df[descriptive_feature] == level]
        entropy_level = compute_impurity(df_feature_level[target], split_criterion)
        entropy_list.append(round(entropy_level, 3))
        weight_level = len(df_feature_level) / len(df)
        weight_list.append(round(weight_level, 3))

    print('impurity of partitions:', entropy_list)
    print('weights of partitions:', weight_list)

    feature_remaining_impurity = np.sum(np.array(entropy_list) * np.array(weight_list))
    print('remaining impurity:', feature_remaining_impurity)
    
    information_gain = target_entropy - feature_remaining_impurity
    print('information gain:', information_gain)
    
    print('====================')

    return(information_gain)

To run the function for Education, Marital Status and Occupation;

In [13]:
split_criteria = 'gini'
for feature in catdata.drop(columns='Annual_Income').columns:
    feature_info_gain = comp_feature_information_gain(catdata, 'Annual_Income', feature, split_criteria)

target feature: Annual_Income
descriptive_feature: Education
split criterion: gini
impurity of partitions: [0.531, 0.375, 0.625]
weights of partitions: [0.4, 0.2, 0.4]
remaining impurity: 0.5374000000000001
information gain: 0.01759999999999995
target feature: Annual_Income
descriptive_feature: Marital_Status
split criterion: gini
impurity of partitions: [0.611, 0.42, 0.375]
weights of partitions: [0.3, 0.5, 0.2]
remaining impurity: 0.4683
information gain: 0.08670000000000005
target feature: Annual_Income
descriptive_feature: Occupation
split criterion: gini
impurity of partitions: [0.5, 0.5, 0.278]
weights of partitions: [0.4, 0.3, 0.3]
remaining impurity: 0.4334
information gain: 0.12160000000000004


Based on the output of the function;

In [14]:

InfGainEducation = 0.017
InfGainMarital_S = 0.087
InfGainOccupation = 0.122


RemEducation = 0.537
RemMarital_S = 0.468
RemOccupation = 0.433

### Age's Information Gain Calculation

We first sort the data in an ascending order of the Age column

In [15]:
datasort = q1.sort_values(by='Age')
datasort

,ID,Age,Education,Marital_Status,Occupation,Annual_Income
2,3,18,high school,never married,agriculture,low
5,6,23,high school,never married,agriculture,low
12,13,23,bachelors,never married,agriculture,low
19,20,25,bachelors,married,transport,high
13,14,25,high school,married,professional,high
15,16,29,bachelors,never married,agriculture,mid
3,4,30,bachelors,married,professional,mid
9,10,33,high school,married,transport,mid
14,15,35,bachelors,married,agriculture,mid
10,11,36,high school,never married,transport,mid


We then examine the points at which the target feature switches from high to low, medium to low, etc. The thresholds for age are `24,27, 38 and 47`. To find out the optimal threshold, the Information Gain needs to be calculated.

To calculate the `Information Gain` of these thresholds, we begin by filtering the data for each threshold and computing the Partition Gini.

#### For Age < 24:

In [16]:
datasort[datasort['Age'] <24]

,ID,Age,Education,Marital_Status,Occupation,Annual_Income
2,3,18,high school,never married,agriculture,low
5,6,23,high school,never married,agriculture,low
12,13,23,bachelors,never married,agriculture,low


In [17]:
PartGL24 = 1 - ((3/3)**2)
PartGL24

0.0

#### For Age >= 24:

In [18]:
datasort[datasort['Age'] >= 24]

,ID,Age,Education,Marital_Status,Occupation,Annual_Income
19,20,25,bachelors,married,transport,high
13,14,25,high school,married,professional,high
15,16,29,bachelors,never married,agriculture,mid
3,4,30,bachelors,married,professional,mid
9,10,33,high school,married,transport,mid
14,15,35,bachelors,married,agriculture,mid
10,11,36,high school,never married,transport,mid
4,5,37,high school,married,agriculture,mid
17,18,37,bachelors,married,professional,mid
18,19,39,high school,divorced,professional,high


In [19]:
PartGG24 = 1- ((5/17)**2 + (12/17)**2 )
PartGG24

0.4152249134948096

In [20]:
PartGG24 = 0.415 # rounded to 3 decimals.

#### For Age < 27:

In [21]:
datasort[datasort['Age'] < 27]

,ID,Age,Education,Marital_Status,Occupation,Annual_Income
2,3,18,high school,never married,agriculture,low
5,6,23,high school,never married,agriculture,low
12,13,23,bachelors,never married,agriculture,low
19,20,25,bachelors,married,transport,high
13,14,25,high school,married,professional,high


In [22]:
PartGL27  = 1 - ((3/5)**2 + (2/5)**2)
PartGL27

0.48

#### For Age >= 27:

In [23]:
datasort[datasort['Age'] >= 27]

,ID,Age,Education,Marital_Status,Occupation,Annual_Income
15,16,29,bachelors,never married,agriculture,mid
3,4,30,bachelors,married,professional,mid
9,10,33,high school,married,transport,mid
14,15,35,bachelors,married,agriculture,mid
10,11,36,high school,never married,transport,mid
4,5,37,high school,married,agriculture,mid
17,18,37,bachelors,married,professional,mid
18,19,39,high school,divorced,professional,high
0,1,39,bachelors,never married,professional,high
7,8,40,doctorate,married,professional,high


In [24]:
PartGG27 = 1-((3/15)**2+(12/15)**2)
PartGG27

0.31999999999999984

In [25]:
PartGG27 = 0.314 # as it is requested, it is rounded for 3 decimals.

#### For  Age < 38 :

In [26]:
datasort[datasort['Age'] < 38]

,ID,Age,Education,Marital_Status,Occupation,Annual_Income
2,3,18,high school,never married,agriculture,low
5,6,23,high school,never married,agriculture,low
12,13,23,bachelors,never married,agriculture,low
19,20,25,bachelors,married,transport,high
13,14,25,high school,married,professional,high
15,16,29,bachelors,never married,agriculture,mid
3,4,30,bachelors,married,professional,mid
9,10,33,high school,married,transport,mid
14,15,35,bachelors,married,agriculture,mid
10,11,36,high school,never married,transport,mid


In [27]:
PartGL38 = 1 - ((3/12)**2 + (2/12)**2 + (7/12)**2)
PartGL38

0.5694444444444444

In [28]:
PartGL38 = 0.569 # as it is requested, it is rounded for 3 decimals.

#### For Age >= 38:

In [29]:
datasort[datasort['Age'] >= 38]

,ID,Age,Education,Marital_Status,Occupation,Annual_Income
18,19,39,high school,divorced,professional,high
0,1,39,bachelors,never married,professional,high
7,8,40,doctorate,married,professional,high
16,17,44,doctorate,divorced,transport,mid
11,12,45,doctorate,married,professional,mid
8,9,46,bachelors,divorced,transport,mid
1,2,50,doctorate,married,professional,mid
6,7,52,high school,divorced,transport,mid


In [30]:
PartGG38 = 1 - ((3/8)**2 + (5/8)**2)
PartGG38

0.46875

In [31]:
PartGG38 = 0.468 # as it is requested, it is rounded for 3 decimals.

#### For Age < 42 threshold:

In [32]:
datasort[datasort['Age'] < 42]

,ID,Age,Education,Marital_Status,Occupation,Annual_Income
2,3,18,high school,never married,agriculture,low
5,6,23,high school,never married,agriculture,low
12,13,23,bachelors,never married,agriculture,low
19,20,25,bachelors,married,transport,high
13,14,25,high school,married,professional,high
15,16,29,bachelors,never married,agriculture,mid
3,4,30,bachelors,married,professional,mid
9,10,33,high school,married,transport,mid
14,15,35,bachelors,married,agriculture,mid
10,11,36,high school,never married,transport,mid


In [33]:
PartGL42 = 1 - ((3/15)**2 + (5/15)**2 + (7/15)**2)
PartGL42

0.6311111111111111

In [34]:
PartGL42 = 0.631 # as it is requested, it is rounded for 3 decimals.

#### For Age >= 42:

In [35]:
datasort[datasort['Age'] >= 42]

,ID,Age,Education,Marital_Status,Occupation,Annual_Income
16,17,44,doctorate,divorced,transport,mid
11,12,45,doctorate,married,professional,mid
8,9,46,bachelors,divorced,transport,mid
1,2,50,doctorate,married,professional,mid
6,7,52,high school,divorced,transport,mid


In [36]:
PartGG42 = 1 - ((5/5)**2)
PartGG42

0.0

Remainders for each threshold equal to the Weight of the partitions which is the number of instances in the each partitions / Total number of instances, computed for each partition. 

$$ Remainder =\sum_l^{} Weight of each Partition * Partition Gini Index$$


#### For Age = 24;

In [37]:
Rem24 = ((3/20)*PartGL24) + ((17/20)*PartGG24)
Rem24

0.35274999999999995

In [38]:
Rem24 = 0.353 # as it is requested, it is rounded for 3 decimals.

#### For Age = 27;

In [39]:
Rem27 = ((5/20)*PartGL27) + ((15/20)*PartGG27)
Rem27

0.3555

In [40]:
Rem27 = 0.36 # as it is requested, it is rounded for 3 decimals.

#### For Age = 38;

In [41]:
Rem38 = ((12/20)*PartGL38) + ((8/20)*PartGG38)
Rem38

0.5286

In [42]:
Rem38 = 0.53 # as it is requested, it is rounded for 3 decimals.

#### For Age = 42;

In [43]:
Rem42 = ((15/20)*PartGL42) + ((5/20)*PartGG42)
Rem42

0.47325

In [44]:
Rem42 = 0.473

$$ Information Gain = Gini Index - Remainder $$

#### For Age = 24;

In [45]:
InfGain24 = 0.555 - Rem24
InfGain24

0.20200000000000007

In [46]:
InfGain24 = 0.202 # as it is requested, it is rounded for 3 decimals.

#### For Age = 27 ;

In [47]:
InfGain27 = 0.555 - Rem27
InfGain27

0.19500000000000006

In [48]:
InfGain27 = 0.195 # as it is requested, it is rounded for 3 decimals.

#### For Age = 38 ;

In [49]:
InfGain38 = 0.555 - Rem38
InfGain38

0.025000000000000022

In [50]:
InfGain38 = 0.026 # as it is requested, it is rounded for 3 decimals.

#### For Age = 42 ;

In [51]:
InfGain42 = 0.555 - Rem42
InfGain42

0.08200000000000007

In [52]:
InfGain42 = 0.082 

Partition sets, Remainder (Rem.), and Information Gain (Info. Gain) for the candidate Age thresholds: 24, 27,38,42;

In [53]:
Age1 = pd.DataFrame(columns=['Feature', 'Threshold','Partition Gini', 'Remainder', 'Information Gain'])
Age1.loc[len(Age1)] = ['Age', '<24', PartGL24, Rem24, InfGain24]
Age1.loc[len(Age1)] = ['Age', '>=24', PartGG24,'' ,'' ]
Age1.loc[len(Age1)] = ['Age', '<27', PartGL27, Rem27, InfGain27]
Age1.loc[len(Age1)] = ['Age', '>=27', PartGG27,'' ,''  ] 
Age1.loc[len(Age1)] = ['Age', '<38', PartGL38, Rem38, InfGain38]
Age1.loc[len(Age1)] = ['Age', '>=38', PartGG38,'' ,''  ]
Age1.loc[len(Age1)] = ['Age', '<42', PartGL42, Rem42, InfGain42]
Age1.loc[len(Age1)] = ['Age', '>=42', PartGG42,'' ,''  ]

Age1

,Feature,Threshold,Partition Gini,Remainder,Information Gain
0,Age,<24,0.000,0.353,0.202
1,Age,>=24,0.415,,
2,Age,<27,0.480,0.36,0.195
3,Age,>=27,0.314,,
4,Age,<38,0.569,0.53,0.026
5,Age,>=38,0.468,,
6,Age,<42,0.631,0.473,0.082
7,Age,>=42,0.000,,


The `df_splits` table shows all the candidate features' thresholds, Information Gains, Remainders and optimal split flags.

In [56]:
df_splits = pd.DataFrame(columns=['Split by Feature','Information Gain', 'Remainder', 'Is Optimal'])
df_splits.loc[len(df_splits)] = ['Age>=24', InfGain24, Rem24, 'TRUE']
df_splits.loc[len(df_splits)] = ['Age>=27', InfGain27,Rem27 ,'FALSE' ]
df_splits.loc[len(df_splits)] = ['Age>=38', InfGain38, Rem38, 'FALSE']
df_splits.loc[len(df_splits)] = ['Age>=42', InfGain42,Rem42 ,'FALSE'  ] 
df_splits.loc[len(df_splits)] = ['Education', InfGainEducation, RemEducation, 'FALSE']
df_splits.loc[len(df_splits)] = ['Marital Status', InfGainMarital_S,RemMarital_S ,'FALSE'  ]
df_splits.loc[len(df_splits)] = ['Occupation', InfGainOccupation,RemOccupation ,'FALSE'  ]

df_splits.round(3)

,Split by Feature,Information Gain,Remainder,Is Optimal
0,Age>=24,0.202,0.353,TRUE
1,Age>=27,0.195,0.360,FALSE
2,Age>=38,0.026,0.530,FALSE
3,Age>=42,0.082,0.473,FALSE
4,Education,0.017,0.537,FALSE
5,Marital Status,0.087,0.468,FALSE
6,Occupation,0.122,0.433,FALSE


From the table, `Age feature's Level== '>=24'` threshold is the optimal split with highest Infomation Gain (0.202)

## Part C

In this Part, we assume 'Education' to be the root node and make the predictions for annual income accordingly. 
We begin by calculating the probabilities of low,mid and high annual income for every level of Education.

In [57]:
datasort[datasort['Education'] == 'high school']

,ID,Age,Education,Marital_Status,Occupation,Annual_Income
2,3,18,high school,never married,agriculture,low
5,6,23,high school,never married,agriculture,low
13,14,25,high school,married,professional,high
9,10,33,high school,married,transport,mid
10,11,36,high school,never married,transport,mid
4,5,37,high school,married,agriculture,mid
18,19,39,high school,divorced,professional,high
6,7,52,high school,divorced,transport,mid


In [58]:
HighSchoolLow = 2/8
HighSchoolLow

0.25

In [59]:
HighSchoolMid = 4/8
HighSchoolMid

0.5

In [60]:
HighSchoolHigh = 2/8
HighSchoolHigh

0.25

In [61]:
q1[q1['Education'] == 'bachelors']

,ID,Age,Education,Marital_Status,Occupation,Annual_Income
0,1,39,bachelors,never married,professional,high
3,4,30,bachelors,married,professional,mid
8,9,46,bachelors,divorced,transport,mid
12,13,23,bachelors,never married,agriculture,low
14,15,35,bachelors,married,agriculture,mid
15,16,29,bachelors,never married,agriculture,mid
17,18,37,bachelors,married,professional,mid
19,20,25,bachelors,married,transport,high


In [62]:
BachelorsLow = 1/8
BachelorsLow

0.125

In [63]:
BachelorsMid = 5/8
BachelorsMid

0.625

In [64]:
BachelorsHigh = 2/8
BachelorsHigh

0.25

In [65]:
q1[q1['Education'] == 'doctorate']

,ID,Age,Education,Marital_Status,Occupation,Annual_Income
1,2,50,doctorate,married,professional,mid
7,8,40,doctorate,married,professional,high
11,12,45,doctorate,married,professional,mid
16,17,44,doctorate,divorced,transport,mid


In [66]:
DoctorateLow = 0

In [67]:
DoctorateMid = 3/4
DoctorateMid

0.75

In [68]:
DoctorateHigh = 1/4
DoctorateHigh

0.25

Combining all the probabilities into one table and defining the predictions based on which probability is higher for what level of income; 

In [69]:
pred = pd.DataFrame(columns=['Leaf Condition', 'Low Income Probability','Mid Income Probability', 'High Income Probability', 'Leaf Prediction'])
pred.loc[len(pred)] = ['Education Level == high school', HighSchoolLow, HighSchoolMid, HighSchoolHigh, 'Mid']
pred.loc[len(pred)] = ['Education Level == bachelors', BachelorsLow,BachelorsMid ,BachelorsHigh, 'Mid']
pred.loc[len(pred)] = ['Education Level == doctorate', DoctorateLow, DoctorateMid, DoctorateHigh, 'Mid']

pred

,Leaf Condition,Low Income Probability,Mid Income Probability,High Income Probability,Leaf Prediction
0,Education Level == high school,0.250,0.500,0.25,Mid
1,Education Level == bachelors,0.125,0.625,0.25,Mid
2,Education Level == doctorate,0.000,0.750,0.25,Mid


In [70]:
df_splits

,Split by Feature,Information Gain,Remainder,Is Optimal
0,Age>=24,0.202,0.353,TRUE
1,Age>=27,0.195,0.360,FALSE
2,Age>=38,0.026,0.530,FALSE
3,Age>=42,0.082,0.473,FALSE
4,Education,0.017,0.537,FALSE
5,Marital Status,0.087,0.468,FALSE
6,Occupation,0.122,0.433,FALSE


In [71]:
pred

,Leaf Condition,Low Income Probability,Mid Income Probability,High Income Probability,Leaf Prediction
0,Education Level == high school,0.250,0.500,0.25,Mid
1,Education Level == bachelors,0.125,0.625,0.25,Mid
2,Education Level == doctorate,0.000,0.750,0.25,Mid
